# PreorderMe - pomocnik do generowania listy zamówień

<img src="logo.png" width="300" height="450">

## Specyfikacja / Specs

* Aplikacja pomaga określić rodzaj i liczbę produktów do zamówienia w danym preorderze
* Tabela do zamówienia składa się z informacji: rodzaj ubrania, numer katalogowy, liczba sztuk (z podziałem na rozmiary)
* Użytkownik wgrywa katalog producenta (.pdf), a aplikacja generuje z niego dane o obecnej kolekcji (dataframe: rodzaj ubrania, numer katalogowy, przeznaczenie, kolor)
* Użytkownik wgrywa historyczny raport (.xls), na podstawie którego aplikacja określa tendencje dla danych produktów
* Użytkownik określa budżet na zamówienie
* Aplikacja generuje raport z listą produktów do zamówienia na bieżący okres w danym budżecie

## Architektura

#### Architektura aplikacji

```plaintext
[INPUT] ─────────► [KATALOG .pdf]
                        │
                        ▼
             [Model klasyfikacji obrazów]
                - Clarifai (ubranie + kolor + ID (OCR))
                - PyMuPDF (zamiana PDF → obrazy)
                        │
                        ▼
           [DataFrame: typ, kolor, ID, rozmiary (?)]

[INPUT] ─────────► [HISTORYCZNE RAPORTY .xls]
                        │
                        ▼
            [Model regresji (np. PyCaret)]
                - prognozy zapotrzebowania
                        │
                        ▼
          [DataFrame: ID, trendy, popyt, sezon]

[INPUT] ─────────► [BUDŻET użytkownika]

                        ▼
   [Algorytm optymalizacji doboru produktów]
        - pulp
        - wybiera najlepsze ID w budżecie
                        │
                        ▼
        [Lista produktów do zamówienia]

                        ▼
     [Generowanie raportu]
          - np. wykresy dla danych produktów, trendy
                        │
                        ▼
[OUTPUT] ◄──────────── [Raport]


#### Uwagi
* najpierw zrobić dla mnie tylko, bez interface'u (żeby nie generować dużej liczby zapytań, tylko pracować na tej tabelce, którą już raz wygenerowałam)
* model klasyfikacji -> zapisać dane w tabelce i już później nie klasyfikować kilka razy
* dane w df, zarówno ze zdj, jak i raportu - wytrenować model raz, zapisać i używać tylko na różnych danych
* budżet sprawdzany za każdym razem algorytmem optymalizacyjnym
* aplikacja już on top of that na samym końcu

## Plan / Taski
* 1 - wgranie katalogu w .pdf + generowanie danych o produktach + wyświetlenie tabeli (na początek dla getrów), sprawdzenie - EDA (describe, czy model się nie pomylił itd.)
* 2 - wgranie historycznego raportu + przygotowanie danych (EDA, zliczenie produktów danej kategorii, usunięcie niepotrzebnych danych) - poczytać o wybieraniu atrybutów/feature selection (!) (np. correlation matrix - module 9, kodowanie rozmiarów, preprocessing, balansowanie, normalzacja)
* 3 - trenowanie modelu regresyjnego (dobór modelu, dobór hyperparameters)
* 4 - użycie modelu do określenia liczby produktów do zakupu dla danego rodzaju (metryki, ocena modelu, ocena dla różnych hiperparametrów, wnioski)
* 5 - wyświetlenie raportu + wykresy (metryki modelu, wykresy ile zamówić, ile było zamówione itd.)
* 6 - uwzględnienie budżetu (np. określenie które produkty są ważniejsze albo że wszystkie są na równi - co chcemy wziąć pod uwagę?)
### Extras
* 7 - Streamlit (wgrywamy jedną stronę .pdf, model regresji mamy)
* 8 - Dane w chmurze (AWS, S3)
* 9 - Porządkowanie danych w bazie danych (PostgreSQL)
* 10 - Automatyczne ETL (AirByte)

## Zależności
* `streamlit` - do budowy interfejsu użytkownika.
* `dotenv` - do przechowywania klucza clarifai.
* `pandas` - do pracy na dataframe.
* `pdf2image` - do przetworzenia .pdf na .jpeg lub .png (zainstalować poppler for Windows)
* `clarifai` - do klasyfikacji ubrań, detekcji, koloru i OCR.
* `pycaret` - do budowy modelu ML.
* `pulp` - do optymalizacji budżetu(?)
* `seaborn` - do wizualizacji wyników.